In [137]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib
%matplotlib inline
import os
import pickle
from collections import deque

In [138]:
def undistort(img):
    img_size = (img.shape[1], img.shape[0])
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(Left.objpoints, Left.imgpoints, img_size,None,None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
  
    return undist

In [139]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [140]:
def calibrate_camera(calibration_folder):

    #Initialize object and image points
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
    
    
    

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = []
    images = glob.glob(calibration_folder)
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp)
    return  objpoints, imgpoints

In [141]:
def magnitude_thresh(img):
    img = np.copy(img)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = gaussian_blur(gray, 5)
    
    #magnitude
    abs_sobel_x = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=11))
    abs_sobel_y = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=11))
    abs_sobelxy = np.sqrt((abs_sobel_x * abs_sobel_x)+(abs_sobel_y * abs_sobel_y))
    scaled_mag = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    mag_binary = np.zeros_like(scaled_mag)
    mag_binary[(scaled_mag >= 60) & (scaled_mag <=255)] = 1
    
    return mag_binary

In [142]:
def direction_thresh(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=11)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=11)
    
    abs_grad_dir = np.arctan2(np.absolute(sobel_y), np.absolute(sobel_x))
    dir_binary =  np.zeros_like(abs_grad_dir)
    dir_binary[(abs_grad_dir >= .7) & (abs_grad_dir <= 1.3)] = 1
                                          
    return dir_binary

In [143]:
def channel_threshold_LUV(img,threshold_range):
    
    l_channel_luv = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)[:,:,0]

    
    l_binary = np.zeros_like(l_channel_luv)
    l_binary[(l_channel_luv >= threshold_range[0]) & (l_channel_luv <= threshold_range[1])] = 1
    
    return l_binary

In [144]:
def channel_threshold_YCrCb(img,threshold_range):
    brightYCB = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)[:,:,0]
    
    ycrcb_binary = np.zeros_like(brightYCB)
    ycrcb_binary[(brightYCB >= threshold_range[0]) & (brightYCB <= threshold_range[1])] = 1
    
    return ycrcb_binary

In [145]:
def channel_threshold_LAB(img,threshold_range):
    
    b_channel_lab = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)[:,:,2]
        
    b_binary = np.zeros_like(b_channel_lab)
    b_binary[(b_channel_lab >= threshold_range[0]) & (b_channel_lab <= threshold_range[1])] = 1
    
    return b_binary

In [146]:
def sobel_x(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    
    abs_sobelx = np.absolute(sobel_x)
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    sobel_x_binary = np.zeros_like(scaled_sobelx)
    sobel_x_binary[(scaled_sobelx >= 15) & (scaled_sobelx <= 255)] = 1
    
    return sobel_x_binary


def sobel_y(img):  
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    
    abs_sobely = np.absolute(sobel_y)
    scaled_sobely = np.uint8(255*abs_sobely/np.max(abs_sobely))
    sobel_y_binary = np.zeros_like(scaled_sobely)
    sobel_y_binary[(scaled_sobely >= 35) & (scaled_sobely <= 255)] = 1
    
    return sobel_y_binary

In [147]:
def color_threshold(img):
    
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel > 60) & (s_channel <= 255)] = 1
    
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >120) & (v_channel <= 255)] = 1 
    
    
    color_binary = np.zeros_like(s_channel)
    color_binary[(s_binary == 1) & (v_binary == 1)] = 1
    return color_binary

In [148]:
def threshold_video_hard(img):
    sobel_x_binary = sobel_x(img)
    sobel_y_binary = sobel_y(img)
    
    mag_binary = magnitude_thresh(img)
    dir_binary = direction_thresh(img)
    
    color_binary = color_threshold(img)
    
    result_lab = channel_threshold_LAB(img, (145,200))
    result_luv = channel_threshold_LUV(img, (215,255))
    
    hard_output = np.zeros_like(sobel_x_binary)
    hard_output[(result_lab == 1) | (result_luv == 1)] = 1
    return hard_output

In [149]:
def transform_prespective(img):
    
    #changed transformed precpective
    img_size = (img.shape[1], img.shape[0])
    src1 = (490, 482) 
    scr2 = (810, 482)
    src3 = (1250, img.shape[0])
    src4 = (0, img.shape[0])
    src_points = np.array([[src1, scr2, src3, src4]]).astype('float32')
    
    dst1 = (0, 0)         
    dst2 = (1280, 0) 
    dst3 = (1250, img.shape[0])
    dst4 = (40, img.shape[0]) 
    dst_points = np.array([[dst1, dst2, dst3, dst4]]).astype('float32')
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    Minv = cv2.getPerspectiveTransform(dst_points, src_points)
    return cv2.warpPerspective(img, M, img_size),Minv

In [150]:
def find_lane_pixels_vid(combined_binary_img):
    
    x, y = np.nonzero(np.transpose(combined_binary_img)) 

    if Left.found == True: # Find the left lane pixels around previous polynomial
        left_x, left_y, Left.found = Left.targeted_search(x, y)
        
    if Right.found == True: # S Find the right lane pixels around previous polynomial
        right_x, right_y, Right.found = Right.targeted_search(x, y)

            
    if Right.found == False: # Perform blind search for right lane lines
        right_x, right_y, Right.found = Right.random_search(x, y, combined_binary_img)
            
    if Left.found == False:# Perform blind search for left lane lines
        left_x, left_y, Left.found = Left.random_search(x, y, combined_binary_img)

    left_y = np.array(left_y).astype(np.float32)
    left_x = np.array(left_x).astype(np.float32)
    right_y = np.array(right_y).astype(np.float32)
    right_x = np.array(right_x).astype(np.float32)
    
    return left_y, left_x, right_y, right_x




In [151]:
def fit_polynomial_vid(left_y, left_x, right_y, right_x):

    left_fit = np.polyfit(left_y, left_x, 2)
    
    # Find intercepts to extend the polynomial to the top and bottom of warped image
    left_x_int, left_top = Left.find_intercepts(left_fit)
    
    # Average intercepts across 'n' frames
    Left.x_int.append(left_x_int)
    Left.top.append(left_top)
    left_x_int = np.mean(Left.x_int)
    left_top = np.mean(Left.top)
    Left.lastx_int = left_x_int
    Left.last_top = left_top
    
    # Add averaged intercepts to current x and y vals
    left_x = np.append(left_x, left_x_int)
    left_y = np.append(left_y, 720)
    left_x = np.append(left_x, left_top)
    left_y = np.append(left_y, 0)
    
    # Sort the detected pixels based on the y-vals
    left_x, left_y = Left.sort_values(left_x, left_y)
    
    Left.X = left_x
    Left.Y = left_y
    
    # Re-calculate polynomial with intercepts and average across 'n' frames
    left_fit = np.polyfit(left_y, left_x, 2)
    Left.fit0.append(left_fit[0])
    Left.fit1.append(left_fit[1])
    Left.fit2.append(left_fit[2])
    left_fit = [np.mean(Left.fit0), np.mean(Left.fit1), np.mean(Left.fit2)]
    
    # Fit polynomial to detected pixels
    left_fit_x = left_fit[0]*left_y**2 + left_fit[1]*left_y + left_fit[2]
    Left.fitx = left_fit_x
    
    # Find right polynomial fit based on detected pixels
    right_fit = np.polyfit(right_y, right_x, 2)

    # Find intercepts to extend the polynomial to the top and bottom of warped image
    right_x_int, right_top = Right.find_intercepts(right_fit)
    
    # Avg. the intercepts across 5 frames
    Right.x_int.append(right_x_int)
    right_x_int = np.mean(Right.x_int)
    Right.top.append(right_top)
    right_top = np.mean(Right.top)
    Right.lastx_int = right_x_int
    Right.last_top = right_top
    right_x = np.append(right_x, right_x_int)
    right_y = np.append(right_y, 720)
    right_x = np.append(right_x, right_top)
    right_y = np.append(right_y, 0)
    
    # Sort right lane pixels
    right_x, right_y = Right.sort_values(right_x, right_y)
    Right.X = right_x
    Right.Y = right_y
    
    # Re-calculate polynomial with intercepts and average across 'n' frames
    right_fit = np.polyfit(right_y, right_x, 2)
    Right.fit0.append(right_fit[0])
    Right.fit1.append(right_fit[1])
    Right.fit2.append(right_fit[2])
    right_fit = [np.mean(Right.fit0), np.mean(Right.fit1), np.mean(Right.fit2)]
    
    # Fit polynomial to detected pixels
    right_fitx = right_fit[0]*right_y**2 + right_fit[1]*right_y + right_fit[2]
    Right.fitx = right_fitx
    
    return right_fitx, left_fit_x


In [152]:
def find_center_offset(binary_warped,left_fit,right_fit):
    
    bottom_y = binary_warped.shape[0] - 1
    bottom_x_left = left_fit[0]*(bottom_y**2) + left_fit[1]*bottom_y + left_fit[2]
    bottom_x_right = right_fit[0]*(bottom_y**2) + right_fit[1]*bottom_y + right_fit[2]
    offset = binary_warped.shape[1]/2 - (bottom_x_left + bottom_x_right)/2

    #pixel meter converstion
    offset *= 3.7/700
    
    return offset

In [153]:
def curve_radius(y_max):

    # meters space conversion
    y_pixel2m = 30/720
    x_pixel2m = 3.7/700

    
    leftx = Left.X
    lefty = Left.Y
    rightx = Right.X
    righty = Right.Y

    # Fit new polynomials to x,y in world space
    left_curve_m = np.polyfit(lefty*y_pixel2m, leftx*y_pixel2m, 2)
    right_curve_m = np.polyfit(righty*x_pixel2m, rightx*x_pixel2m, 2)
    
    # Calculate the new radius for the curve
    left_curve_radius = ((1 + (2*left_curve_m[0]*y_max*y_pixel2m + left_curve_m[1])**2)**1.5) / np.absolute(2*left_curve_m[0])
    right_curve_radius = ((1 + (2*right_curve_m[0]*y_max*y_pixel2m + right_curve_m[1])**2)**1.5) / np.absolute(2*right_curve_m[0])


    return left_curve_radius, right_curve_radius

In [154]:
def Lane_Area_Vid(binary_warped, undistorted, left_fitx, right_fitx, m_inv,avg_radius):

    #create lane area mask
    temp_lane_area = np.zeros_like(binary_warped).astype(np.uint8)
    color_warped = np.dstack((temp_lane_area, temp_lane_area, temp_lane_area))

     #find offset
    center_offset = find_center_offset(binary_warped, left_fitx,right_fitx)
    

    pts_left = np.array([np.flipud(np.transpose(np.vstack([left_fitx, Left.Y])))])
    pts_right = np.array([np.transpose(np.vstack([right_fitx, Right.Y]))])
    pts = np.hstack((pts_left, pts_right))
    cv2.polylines(color_warped, np.int_([pts]), isClosed=False, color=(0,0,0), thickness = 40)

     #fill green color
    cv2.fillPoly(color_warped, np.int_([pts]), (0,255, 0))

    # Highlight lane area mask in undistorted
    inverse_transformed = cv2.warpPerspective(color_warped, m_inv, (undistorted.shape[1], undistorted.shape[0]))
    
    # Combine the result with the original image
    lane_area = cv2.addWeighted(undistorted, 1, inverse_transformed, 0.5, 0)

    # Write text in image
    radius_text = 'Radius of curve is ' + str(round(avg_radius, 2)) + ' m' 
                      
    lane_area = cv2.putText(lane_area, radius_text, (450,40), 0, 1, (0,0,0), 2, cv2.LINE_AA)
    
    return lane_area

In [155]:
class Line():
    
    def __init__(self):
        self.X = None # fit values position in xaxis
        self.Y = None # fit values position in yaxis
        calibration_folder ='camera_cal/calibration*.jpg'
        self.objpoints, self.imgpoints = calibrate_camera(calibration_folder)
        self.found = False  
        
        # Store recent polynomial coefficients for averaging across frames
        self.fit0 = deque(maxlen=10)
        self.fit1 = deque(maxlen=10)
        self.fit2 = deque(maxlen=10)
        
        self.fitx = None
        
        # Keep recent 'x' intercepts avg. across frames
        self.x_int = deque(maxlen=10)
        self.top = deque(maxlen=10)
        
        
    def targeted_search(self, x, y):
        x_vals = []
        y_vals = []
        if self.found == True: 
            i = 720
            j = 630
            while j >= 0:
                y_val = np.mean([i,j])
                x_val = (np.mean(self.fit0))*y_val**2 + (np.mean(self.fit1))*y_val + (np.mean(self.fit2))
                x_idx = np.where((((x_val - 10) < x)&(x < (x_val + 10))&((y > j) & (y < i))))
                x_window, y_window = x[x_idx], y[x_idx]
                if np.sum(x_window) != 0:
                    np.append(x_vals, x_window)
                    np.append(y_vals, y_window)
                i -= 90
                j -= 90
        if np.sum(x_vals) == 0: 
            self.found = False
        return x_vals, y_vals, self.found
    
    def random_search(self, x, y, image):
        x_vals = []
        y_vals = []
        if self.found == False: 
            i = 720
            j = 630
            while j >= 0:
                histogram = np.sum(image[j:i,:], axis=0)
                if self == Right:
                    peak = np.argmax(histogram[640:]) + 640
                else:
                    peak = np.argmax(histogram[:640])
                x_idx = np.where((((peak - 25) < x)&(x < (peak + 25))&((y > j) & (y < i))))
                x_window, y_window = x[x_idx], y[x_idx]
                if np.sum(x_window) != 0:
                    x_vals.extend(x_window)
                    y_vals.extend(y_window)
                i -= 90
                j -= 90
        if np.sum(x_vals) > 0:
            self.found = True
        else:
            y_vals = self.Y
            x_vals = self.X
        return x_vals, y_vals, self.found
    
    
    def find_intercepts(self, polynomial):
        bottom = polynomial[0]*720**2 + polynomial[1]*720 + polynomial[2]
        top = polynomial[0]*0**2 + polynomial[1]*0 + polynomial[2]
        return bottom, top
    
    def sort_values(self, x_vals, y_vals):
        sorted_index = np.argsort(y_vals)
        sorted_y_vals = y_vals[sorted_index]
        sorted_x_vals = x_vals[sorted_index]
        return sorted_x_vals, sorted_y_vals
        
        

In [156]:

#to initialize values and later pass through for next frame
Left = Line()
Right = Line()


In [157]:
# Pipeline for Video Processing
def process_image(image):
    
    #calibrate and undistory image based on camera parameters
    undistorted = undistort(image)
    
    #prespective transformation is applied to get birds eye view
    warped_image, m_inv = transform_prespective(undistorted)

    #thresholded image to extract lane lines from frame
    thresholded = threshold_video_hard(warped_image)
    
    # Detect the lane pixels in the image
    left_y, left_x, right_y, right_x =  find_lane_pixels_vid(thresholded)
            
    # Find left & right polynomial fit based on detected pixels
    right_fitx, left_fitx = fit_polynomial_vid(left_y, left_x, right_y, right_x)

    # Calculate radius of curvature for each lane (in meters)
    left_curve_radius, right_curve_radius = curve_radius(undistorted.shape[0]-1)
    avg_radius = (left_curve_radius + right_curve_radius)/2
    
    ##highlight lane area 
    out_img = Lane_Area_Vid(thresholded, undistorted, left_fitx, right_fitx, m_inv,avg_radius)
    
    return out_img

In [158]:
from moviepy.editor import VideoFileClip

video_output = 'Output_challenge_cruv.mp4'
clip = VideoFileClip('challenge_video.mp4')

output_clip = clip.fl_image(process_image)
%time output_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video Output_challenge_cruv.mp4
[MoviePy] Writing video Output_challenge_cruv.mp4



100%|██████████| 485/485 [07:05<00:00,  1.11it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Output_challenge_cruv.mp4 

CPU times: user 6min 17s, sys: 1.17 s, total: 6min 18s
Wall time: 7min 8s
